In [17]:
import copy
import json
import logging
from collections import defaultdict
from typing import Any, Dict, List, Optional, Sequence, Tuple, TypeVar, Union
import numpy as np
import math
import numpy
import torch
from torch.autograd import Variable

In [18]:
# Inputs to the function

# logits shape torch.Size([4, 16, 16])
# targets shape torch.Size([4, 16])
# weights shape torch.Size([4, 16])


# weights tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
#         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
#         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
#         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

# targets tensor([[ 9,  9,  3,  6,  7,  1,  0,  4,  0,  4,  0,  5,  0,  0,  0,  0],
#         [ 7,  0,  6,  9,  9,  1,  5,  2,  3,  1,  8,  1,  5,  0,  0,  0],
#         [ 1,  8,  1,  0,  5,  3,  2,  3,  0,  9,  3,  2,  4,  7,  0,  5],
#         [ 9,  7,  9,  1,  0,  6,  2,  9, 11,  2,  4, 10, 11,  5,  0,  0]])

In [20]:
batch_size = 4
nb_classes = 16
seq_len = 16

In [27]:
logits = torch.randn(batch_size, seq_len, nb_classes)
print(" == logits == \n", logits)
print(logits.shape)

# don't have any 0s in targets as it effects weights of func 2.
targets = torch.randint(1, 16, size=(batch_size, seq_len))
#targets = torch.randn(batch_size, seq_len)
print(" == targets == \n", targets)
print(targets.shape)

weights = ([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

# weights = ([[1., 1., 1., 0.],
#         [1., 1., 1., 0.]])

weights = torch.FloatTensor(weights)
print(" == weights == \n", weights)
print(weights.shape)

 == logits == 
 tensor([[[-0.0454, -1.6380,  2.3542,  ..., -0.9927,  0.5039, -1.6639],
         [-2.0609, -0.1287,  1.5118,  ...,  1.4356, -0.2556,  0.1615],
         [-0.5814,  0.3074,  0.1140,  ..., -0.5420,  0.2809, -0.6194],
         ...,
         [ 1.0790, -1.2793, -0.8957,  ...,  0.1858, -1.5915,  1.0700],
         [ 0.3916, -0.0579, -0.3122,  ...,  0.7189,  0.1638, -1.3539],
         [ 0.1964, -0.7673,  1.1060,  ...,  0.2558, -2.2480, -1.2574]],

        [[-0.4934,  2.6456,  1.6421,  ...,  0.0258,  1.3745,  0.4510],
         [ 1.0148, -0.2019,  1.1736,  ..., -0.9428,  0.6456,  0.2396],
         [-0.8986, -0.0579, -0.8005,  ..., -0.8566, -1.2162,  1.1290],
         ...,
         [ 1.5684,  1.7940,  0.9422,  ..., -0.2551,  1.0192, -0.2893],
         [ 0.3011,  0.8326,  1.9589,  ..., -0.0898, -0.1556,  0.6500],
         [ 0.2842, -0.2804, -0.1563,  ..., -1.1728, -0.3659,  0.7056]],

        [[ 1.7123,  0.0872, -1.6509,  ..., -0.7854, -1.4560, -1.4729],
         [ 1.4867, -0.2596,  

## Case 1: original function

In [38]:
def sequence_cross_entropy_with_logits(
    logits: torch.FloatTensor,
    targets: torch.LongTensor,
    weights: torch.FloatTensor,
    average: str = "batch",
    label_smoothing: float = None,
    gamma: float = None,
    alpha: Union[float, List[float], torch.FloatTensor] = None,
) -> torch.FloatTensor:
    """
    Computes the cross entropy loss of a sequence, weighted with respect to
    some user provided weights. Note that the weighting here is not the same as
    in the `torch.nn.CrossEntropyLoss()` criterion, which is weighting
    classes; here we are weighting the loss contribution from particular elements
    in the sequence. This allows loss computations for models which use padding.
    # Parameters
    logits : `torch.FloatTensor`, required.
        A `torch.FloatTensor` of size (batch_size, sequence_length, num_classes)
        which contains the unnormalized probability for each class.
    targets : `torch.LongTensor`, required.
        A `torch.LongTensor` of size (batch, sequence_length) which contains the
        index of the true class for each corresponding step.
    weights : `torch.FloatTensor`, required.
        A `torch.FloatTensor` of size (batch, sequence_length)
    average: str, optional (default = "batch")
        If "batch", average the loss across the batches. If "token", average
        the loss across each item in the input. If `None`, return a vector
        of losses per batch element.
    label_smoothing : `float`, optional (default = None)
        Whether or not to apply label smoothing to the cross-entropy loss.
        For example, with a label smoothing value of 0.2, a 4 class classification
        target would look like `[0.05, 0.05, 0.85, 0.05]` if the 3rd class was
        the correct label.
    gamma : `float`, optional (default = None)
        Focal loss[*] focusing parameter `gamma` to reduces the relative loss for
        well-classified examples and put more focus on hard. The greater value
        `gamma` is, the more focus on hard examples.
    alpha : `float` or `List[float]`, optional (default = None)
        Focal loss[*] weighting factor `alpha` to balance between classes. Can be
        used independently with `gamma`. If a single `float` is provided, it
        is assumed binary case using `alpha` and `1 - alpha` for positive and
        negative respectively. If a list of `float` is provided, with the same
        length as the number of classes, the weights will match the classes.
        [*] T. Lin, P. Goyal, R. Girshick, K. He and P. Dollár, "Focal Loss for
        Dense Object Detection," 2017 IEEE International Conference on Computer
        Vision (ICCV), Venice, 2017, pp. 2999-3007.
    # Returns
    A torch.FloatTensor representing the cross entropy loss.
    If `average=="batch"` or `average=="token"`, the returned loss is a scalar.
    If `average is None`, the returned loss is a vector of shape (batch_size,).
    """
    if average not in {None, "token", "batch"}:
        raise ValueError("Got average f{average}, expected one of None, 'token', or 'batch'")

    # make sure weights are float
    weights = weights.float()
    # sum all dim except batch
    non_batch_dims = tuple(range(1, len(weights.shape)))
    # shape : (batch_size,)
    weights_batch_sum = weights.sum(dim=non_batch_dims)
    # shape : (batch * sequence_length, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    # shape : (batch * sequence_length, num_classes)
    log_probs_flat = torch.nn.functional.log_softmax(logits_flat, dim=-1)
    print("== log probs flat == \n", log_probs_flat)
    print(log_probs_flat.shape)
    
    # shape : (batch * max_len, 1)
    targets_flat = targets.view(-1, 1).long()
    print("== targets flat == \n", targets_flat)
    print(targets_flat.shape)
    # focal loss coefficient
    if gamma:
        # shape : (batch * sequence_length, num_classes)
        probs_flat = log_probs_flat.exp()
        # shape : (batch * sequence_length,)
        probs_flat = torch.gather(probs_flat, dim=1, index=targets_flat)
        # shape : (batch * sequence_length,)
        focal_factor = (1.0 - probs_flat) ** gamma
        # shape : (batch, sequence_length)
        focal_factor = focal_factor.view(*targets.size())
        weights = weights * focal_factor

    if alpha is not None:
        # shape : () / (num_classes,)
        if isinstance(alpha, (float, int)):

            # shape : (2,)
            alpha_factor = torch.tensor(
                [1.0 - float(alpha), float(alpha)], dtype=weights.dtype, device=weights.device
            )

        elif isinstance(alpha, (list, numpy.ndarray, torch.Tensor)):

            # shape : (c,)
            alpha_factor = torch.tensor(alpha, dtype=weights.dtype, device=weights.device)

            if not alpha_factor.size():
                # shape : (1,)
                alpha_factor = alpha_factor.view(1)
                # shape : (2,)
                alpha_factor = torch.cat([1 - alpha_factor, alpha_factor])
        else:
            raise TypeError(
                ("alpha must be float, list of float, or torch.FloatTensor, {} provided.").format(
                    type(alpha)
                )
            )
        # shape : (batch, max_len)
        alpha_factor = torch.gather(alpha_factor, dim=0, index=targets_flat.view(-1)).view(
            *targets.size()
        )
        weights = weights * alpha_factor

    if label_smoothing is not None and label_smoothing > 0.0:
        num_classes = logits.size(-1)
        smoothing_value = label_smoothing / num_classes
        # Fill all the correct indices with 1 - smoothing value.
        one_hot_targets = torch.zeros_like(log_probs_flat).scatter_(
            -1, targets_flat, 1.0 - label_smoothing
        )
        smoothed_targets = one_hot_targets + smoothing_value
        negative_log_likelihood_flat = -log_probs_flat * smoothed_targets
        negative_log_likelihood_flat = negative_log_likelihood_flat.sum(-1, keepdim=True)
    else:
        # Contribution to the negative log likelihood only comes from the exact indices
        # of the targets, as the target distributions are one-hot. Here we use torch.gather
        # to extract the indices of the num_classes dimension which contribute to the loss.
        # shape : (batch * sequence_length, 1)
        negative_log_likelihood_flat = -torch.gather(log_probs_flat, dim=1, index=targets_flat)
        print("== negative_log_likelihood_flat == \n", negative_log_likelihood_flat)
        print(negative_log_likelihood_flat.shape)
    # shape : (batch, sequence_length)
    negative_log_likelihood = negative_log_likelihood_flat.view(*targets.size())
    print("== negative_log_likelihood == \n", negative_log_likelihood)
    # shape : (batch, sequence_length)
    negative_log_likelihood = negative_log_likelihood * weights
    print("== negative_log_likelihood * weights == \n", negative_log_likelihood)

    if average == "batch":
        # shape : (batch_size,)
        print("== nll summmed == \n", negative_log_likelihood.sum(non_batch_dims))
        print("== weights_batch_sum == \n", weights_batch_sum)
        per_batch_loss = negative_log_likelihood.sum(non_batch_dims) / (weights_batch_sum + 1e-13)
        
        num_non_empty_sequences = (weights_batch_sum > 0).float().sum() + 1e-13
        print("== num_non_empty_sequences == \n", num_non_empty_sequences)
        print(per_batch_loss.sum() / num_non_empty_sequences)
        return per_batch_loss.sum() / num_non_empty_sequences
    elif average == "token":
        return negative_log_likelihood.sum() / (weights_batch_sum.sum() + 1e-13)
    else:
        # shape : (batch_size,)
        per_batch_loss = negative_log_likelihood.sum(non_batch_dims) / (weights_batch_sum + 1e-13)
        return per_batch_loss

In [39]:
# get the loss 
loss = sequence_cross_entropy_with_logits(logits, targets, weights)
print(loss)

== log probs flat == 
 tensor([[-3.3966, -4.9893, -0.9970,  ..., -4.3439, -2.8473, -5.0151],
        [-5.5061, -3.5739, -1.9334,  ..., -2.0096, -3.7008, -3.2837],
        [-4.1109, -3.2221, -3.4155,  ..., -4.0715, -3.2486, -4.1489],
        ...,
        [-4.1386, -3.1759, -4.4606,  ..., -2.8018, -2.7199, -3.8283],
        [-2.7388, -3.1461, -3.6381,  ..., -3.3881, -4.3178, -2.9086],
        [-4.0282, -3.9332, -3.1941,  ..., -3.3195, -3.2721, -4.6192]])
torch.Size([64, 16])
== targets flat == 
 tensor([[15],
        [ 3],
        [ 7],
        [15],
        [14],
        [ 1],
        [14],
        [ 6],
        [ 2],
        [11],
        [10],
        [ 8],
        [ 3],
        [ 6],
        [14],
        [ 3],
        [12],
        [ 5],
        [11],
        [14],
        [ 6],
        [15],
        [ 8],
        [ 1],
        [14],
        [ 5],
        [ 9],
        [ 1],
        [13],
        [14],
        [ 7],
        [ 4],
        [ 4],
        [ 3],
        [11],
        [ 4

In [119]:
## Case 2: my custom function

def mask_tensor(tensor):
    return (tensor != 0).float()

def nested_sequence_cross_entropy_with_logits(
    logits: torch.FloatTensor,
    targets: torch.LongTensor,
    weights: torch.FloatTensor,
    average: str = "batch",
    label_smoothing: float = None,
    gamma: float = None,
    alpha: Union[float, List[float], torch.FloatTensor] = None,
) -> torch.FloatTensor:
    """

    """
    if average not in {None, "token", "batch"}:
        raise ValueError("Got average f{average}, expected one of None, 'token', or 'batch'")
        
    # weights mask, not sure if we need these this time..   
    weights = weights.float()
    print("weights", weights)
    # sum all dim except batch
    non_batch_dims = tuple(range(1, len(weights.shape)))
    
    print("non batch dims", non_batch_dims)
    # shape : (batch_size,)
    # sum weights along the row-dimension
    # correspons to number of active elements for each item in the batch
    weights_batch_sum = weights.sum(dim=non_batch_dims)
    print("weights batch sum", weights_batch_sum)
    
    
    batch_size = targets.shape[0]    
    seq_len = targets.shape[1]
    nb_labels = targets.shape[-1]

    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    #print("== logits flat == \n", logits_flat)
    
    # log_probs_flat: (batch * max_len, num_classes)
    log_probs_flat = torch.nn.functional.log_softmax(logits_flat, dim=-1)
    print("== log probs flat == \n", log_probs_flat)
    
    # target_flat: (batch * max_len, 1)
    #target_flat = target.view(-1, 1)
    targets_flat = targets.view(-1, nb_labels)
    print("== targets flat == \n", targets_flat)
    
    # losses_flat: (batch * max_len, 1)
    # dimensions need to be the same size except for the dim you are gathering on
    # gathers from log probs the target index (row dim)
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=targets_flat)
    print("== losses flat == \n", losses_flat)
    
    # losses: (batch, max_len)
    losses = losses_flat.view(*targets.size())
    print("== losses == \n", losses)

    # return the tensor where 1s correspond to active elements and 0s are padded elements
    tensor_mask = mask_tensor(targets)
    
    # multiply losses by label mask
    masked_losses = losses * tensor_mask
    print("== masked losses == \n", masked_losses)
    
    # find the number of 0 elements in losses
    # a 0 here means all elements were activate and n corresponds to how many zeros were in the row
    num_zero_labels = (masked_losses == 0).sum(-1)
    print("== num zero labels == \n", num_zero_labels)
    
    # possible number of values, multiply by constant
    max_possible_labels = torch.ones(size=(batch_size, seq_len)) * nb_labels
    
    # subtract the number of possible active elements by the number of 0-padded elements
    num_active_labels = max_possible_labels - num_zero_labels
    print("== num active labels == \n", num_active_labels)
  
    # add loss along row-dim then divide by the number of active elements per row
    per_batch_loss = masked_losses.sum(-1) / (num_active_labels + 1e-13)
    print("== per_batch_loss == \n", per_batch_loss)
    
    # at this point, we are back to 2d shape as we have summed/divided by the number of labels
    # we can use the regular weights values
    # multiply losses by token-level mask
    masked_per_batch_loss = per_batch_loss * weights
    print("== masked_per_batch_loss == \n", masked_per_batch_loss)
    
    
    
    # shape : (batch_size,)
    print("== masked_per_batch_loss_summmedd == \n", masked_per_batch_loss.sum(non_batch_dims))
    # weights added up
    print("== weights_batch_sum == \n", weights_batch_sum)
    
    per_batch_loss = masked_per_batch_loss.sum(non_batch_dims) / (weights_batch_sum + 1e-13)

    num_non_empty_sequences = (weights_batch_sum > 0).float().sum() + 1e-13
    print("== num_non_empty_sequences == \n", num_non_empty_sequences)
    print(per_batch_loss.sum() / num_non_empty_sequences)
    return per_batch_loss.sum() / num_non_empty_sequences
    

In [120]:
def expand_targets(targets):
    # need to add a 3rd dim and put the label there and pad it with 0s
    expanded_targets = targets.unsqueeze(-1)
    expanded_targets = expanded_targets.tolist()
    
    padding = [0,0]

    for item in expanded_targets:
        for labels in item:
            labels.extend(padding)
    
    #print(expanded_targets)
    expanded_targets = torch.LongTensor(expanded_targets)
    return expanded_targets

In [121]:
## Case 2
print(targets)
print(targets.shape)

tensor([[15,  3,  7, 15, 14,  1, 14,  6,  2, 11, 10,  8,  3,  6, 14,  3],
        [12,  5, 11, 14,  6, 15,  8,  1, 14,  5,  9,  1, 13, 14,  7,  4],
        [ 4,  3, 11,  4, 12, 15,  1,  7,  1,  6,  1,  6, 15,  9,  1, 15],
        [ 1, 13, 11,  8,  9, 12,  1,  5,  3, 11, 12, 10,  6, 10, 10, 11]])
torch.Size([4, 16])


In [122]:
expanded_targets = expand_targets(targets)
#print(expanded_targets)
#expanded_targets = torch.LongTensor(expand_targets)

In [123]:
loss2 = nested_sequence_cross_entropy_with_logits(logits, expanded_targets, weights)

weights tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
non batch dims (1,)
weights batch sum tensor([13., 13., 14., 16.])
== log probs flat == 
 tensor([[-3.3966, -4.9893, -0.9970,  ..., -4.3439, -2.8473, -5.0151],
        [-5.5061, -3.5739, -1.9334,  ..., -2.0096, -3.7008, -3.2837],
        [-4.1109, -3.2221, -3.4155,  ..., -4.0715, -3.2486, -4.1489],
        ...,
        [-4.1386, -3.1759, -4.4606,  ..., -2.8018, -2.7199, -3.8283],
        [-2.7388, -3.1461, -3.6381,  ..., -3.3881, -4.3178, -2.9086],
        [-4.0282, -3.9332, -3.1941,  ..., -3.3195, -3.2721, -4.6192]])
== targets flat == 
 tensor([[15,  0,  0],
        [ 3,  0,  0],
        [ 7,  0,  0],
        [15,  0,  0],
        [14,  0,  0],
        [ 1,  0,  0],
        [14,  0,  